In [66]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from scipy.optimize import minimize

In [67]:
np.random.seed(77)

In [68]:
tickers = [f"Ticker_{i}" for i in range(3)]
num_values = 10**6
annual_stock_price_df = pd.DataFrame(np.random.uniform(-10, 10, size=(num_values, len(tickers))), columns=tickers)

annual_stock_price_df.head(3)

,Ticker_0,Ticker_1,Ticker_2
0,8.382181,2.843912,5.074245
1,-7.213709,-8.253609,5.760041
2,-3.476981,0.821356,-5.195296


In [69]:
def create_functional_1(random_values):
    def wrapper(weights):
        weighted_annual_ratio_sum_df = random_values.mul(weights).sum(axis=1)

        weighted_expectation = weighted_annual_ratio_sum_df.mean()
        weighted_variance = weighted_annual_ratio_sum_df.var(ddof=0)
        return weighted_expectation / np.sqrt(weighted_variance)

    return wrapper


def create_functional_2(random_values):
    cov_matrix = random_values.cov(ddof=0).values
    np.fill_diagonal(cov_matrix, 0)
    
    def wrapper(weights):
        numerator = weights @ random_values.mean().values
        denominator_var = (weights ** 2) @ random_values.var(ddof=0).values
        denominator_cov = (weights @ cov_matrix) @ weights
        return numerator / np.sqrt(denominator_var + denominator_cov)

    return wrapper

In [70]:
def constraint(x):
    return sum(x) - 1

con = {"type": "eq", "fun": constraint}
bounds = [(0, 1) for _ in range(len(tickers))]

functional_1 = create_functional_1(annual_stock_price_df)
functional_2 = create_functional_2(annual_stock_price_df)

minimization_functional_1 = lambda weights: - functional_1(weights)
minimization_functional_2 = lambda weights: - functional_2(weights)

In [71]:
def generate_random_weights(vector_size):
    weights = np.random.rand(vector_size)
    weights /= np.sum(weights)
    return weights


def random_search_for_initial_weights(functional_to_minimize, n_tries, vector_size):
    best_func_value = np.inf
    for _ in tqdm(range(n_tries)):
        weights = generate_random_weights(vector_size)
        res = minimize(functional_to_minimize, weights, constraints=con, bounds=bounds)
        if res.fun < best_func_value:
            best_func_value = res.fun

    return best_func_value

### 3 columns

In [72]:
func_1_value = random_search_for_initial_weights(minimization_functional_1, 3, vector_size=len(tickers))
print("functional_1:", func_1_value)

100%|██████████| 3/3 [00:10<00:00,  3.54s/it]

functional_1: -0.001277300332977002


In [73]:
func_2_value = random_search_for_initial_weights(minimization_functional_2, 3, vector_size=len(tickers))
print("functional_2:", func_2_value)

100%|██████████| 3/3 [00:04<00:00,  1.66s/it]

functional_2: -0.0012674360557667008


### 10 columns

In [74]:
tickers = [f"Ticker_{i}" for i in range(10)]
annual_stock_price_df = pd.DataFrame(np.random.uniform(-10, 10, size=(num_values, len(tickers))), columns=tickers)

bounds = [(0, 1) for _ in range(len(tickers))]

functional_1 = create_functional_1(annual_stock_price_df)
functional_2 = create_functional_2(annual_stock_price_df)

minimization_functional_1 = lambda weights: - functional_1(weights)
minimization_functional_2 = lambda weights: - functional_2(weights)

annual_stock_price_df.head(3)

,Ticker_0,Ticker_1,Ticker_2,Ticker_3,Ticker_4,Ticker_5,Ticker_6,Ticker_7,Ticker_8,Ticker_9
0,-2.981269,-0.707952,0.661833,-4.339718,-2.669090,-0.401840,9.874176,8.561488,-6.997616,-1.328227
1,3.071128,-4.589168,1.113575,3.379856,-4.138881,8.614578,0.862149,0.651832,-6.883063,5.041497
2,-6.881038,-8.411451,-0.144322,-3.098648,9.675710,-2.602158,2.708409,-3.566908,2.083498,7.595213


In [75]:
func_1_value = random_search_for_initial_weights(minimization_functional_1, 3, vector_size=len(tickers))
print("functional_1:", func_1_value)

100%|██████████| 3/3 [01:08<00:00, 22.85s/it]

functional_1: -0.002422184991851932


In [76]:
func_2_value = random_search_for_initial_weights(minimization_functional_2, 3, vector_size=len(tickers))
print("functional_2:", func_2_value)

100%|██████████| 3/3 [01:13<00:00, 24.64s/it]

functional_2: -0.0024222152154985822
